In [45]:
import pandas as pd
import ast

In [62]:
trainingdata = pd.read_csv('trainingdata.csv')

In [63]:
trainingdata['categories'] = trainingdata['categories'].apply(ast.literal_eval)
expanded = trainingdata.explode('categories')
one_hot = pd.get_dummies(expanded['categories'])

# Group by the original DataFrame index and take the maximum to condense back to the original shape
resultC = trainingdata.drop('categories', axis=1).join(one_hot.groupby(level=0).max())

In [64]:
trainingdata['mechanics'] = trainingdata['mechanics'].apply(ast.literal_eval)
expanded = trainingdata.explode('mechanics')
one_hot = pd.get_dummies(expanded['mechanics'])

# Group by the original DataFrame index and take the maximum to condense back to the original shape
resultM = trainingdata.drop('mechanics', axis=1).join(one_hot.groupby(level=0).max())

In [65]:
final_result = pd.concat([resultC, resultM], axis=1)

In [67]:
trainingdata = final_result

In [ ]:
#trainingdata.to_csv('cleantraining.csv')

Now for testing data

In [69]:
testingdata = pd.read_csv('testingdata.csv')

In [70]:
testingdata['categories'] = testingdata['categories'].apply(ast.literal_eval)
expanded = testingdata.explode('categories')
one_hot = pd.get_dummies(expanded['categories'])

# Group by the original DataFrame index and take the maximum to condense back to the original shape
resultC = testingdata.drop('categories', axis=1).join(one_hot.groupby(level=0).max())

In [71]:
testingdata['mechanics'] = testingdata['mechanics'].apply(ast.literal_eval)
expanded = testingdata.explode('mechanics')
one_hot = pd.get_dummies(expanded['mechanics'])

# Group by the original DataFrame index and take the maximum to condense back to the original shape
resultM = testingdata.drop('mechanics', axis=1).join(one_hot.groupby(level=0).max())

In [72]:
final_result = pd.concat([resultC, resultM], axis=1)

In [ ]:
#final_result.to_csv('cleantesting.csv')

In [81]:
testingdata = final_result

Need a bit more cleaning done

In [95]:
trainingdata = trainingdata.loc[:,~trainingdata.columns.duplicated()].copy()
testingdata = testingdata.loc[:,~testingdata.columns.duplicated()].copy()

In [97]:
trainmissing =set(testingdata.columns) - set(trainingdata.columns)

In [98]:
testmissing = set(trainingdata.columns) - set(testingdata.columns)

In [99]:
for col in trainmissing:
    trainingdata[col] = False
    
for col in testmissing:
    testingdata[col] = False

In [102]:
set(testingdata.columns)-set(trainingdata.columns)

set()

In [103]:
set(trainingdata.columns)-set(testingdata.columns)

set()

Now I will feature engineer from the description

In [107]:
import re

In [106]:
trainingdata['DescLen']=trainingdata['descriptions'].str.len()
testingdata['DescLen']=testingdata['descriptions'].str.len()

In [113]:
trainingdata['Sentences']=trainingdata['descriptions'].apply(lambda x : len(re.findall(r'[.?!]', x)))
testingdata['Sentences']=testingdata['descriptions'].apply(lambda x : len(re.findall(r'[.?!]', x)))

In [114]:
def unique_word_count(text):
    if pd.isna(text) or text.strip() == "":
        return 0
    # Extract words (alphanumeric sequences)
    words = re.findall(r'\b\w+\b', text.lower())  # Case insensitive
    return len(set(words))  # Count unique words

# Function to calculate special character count
def special_char_count(text):
    if pd.isna(text) or text.strip() == "":
        return 0
    # Count non-alphanumeric and non-whitespace characters
    return len(re.findall(r'[^\w\s]', text))

In [117]:
testingdata['specialChar']=testingdata['descriptions'].apply(special_char_count)
testingdata['uniqueWords']=testingdata['descriptions'].apply(unique_word_count)

trainingdata['specialChar']=trainingdata['descriptions'].apply(special_char_count)
trainingdata['uniqueWords']=trainingdata['descriptions'].apply(unique_word_count)

In [ ]:
#testingdata.to_csv('cleantesting.csv')
#trainingdata.to_csv('cleantraining.csv')

In [119]:
testingdata

,Unnamed: 0,id,name,yearpublished,bayesaverage,avgplaytime,complexity,descriptions,minplayers,maxplayers,...,"I Cut, You Choose",Turn Order: Random,Action Timer,Automatic Resource Growth,Tags,Cube Tower,DescLen,Sentences,specialChar,uniqueWords
0,14562,275087,Auf der Walz,2019,5.53297,150,2.6667,Auf der Walz is a game that has the &quot;jour...,2,4,...,False,False,False,False,False,False,1352,11,89,135
1,24735,220741,Yummy Yummy Pancake,2017,5.48779,15,1.0000,"In Yummy Yummy Pancake, you're trying to ident...",2,4,...,False,False,False,False,False,False,942,9,53,88
2,16994,144993,Clusterfight!,2014,5.52187,45,1.0000,Clusterfight is a party game of hypothetical b...,3,12,...,False,False,False,False,False,False,839,11,52,99
3,22151,368898,Rhett & Link's We're Still Good Game,2022,5.50238,30,1.5000,FROM THE GOOD MYTHICAL MORNING DUO: Dive into ...,4,12,...,False,False,False,False,False,False,1326,14,90,101
4,8378,294113,Imperial Tide: The Great War 1914-1918,2022,5.60447,240,2.3333,Imperial Tide is a 2-player strategic level wa...,1,2,...,False,False,False,False,False,False,496,4,29,60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,16878,382801,Grass is Greener,2023,5.52244,45,1.6000,Strategize and scheme against your neighbors i...,2,6,...,False,False,False,False,False,False,626,7,42,66
496,25542,146596,Logo Party,2013,5.47962,60,1.2000,Official Game Description:&#10;&#10;It's not w...,4,0,...,False,False,False,False,False,False,521,11,62,63
497,17208,123689,"Fornovo 1495: Dawn of the Italian Wars, 1495-1525",2016,5.52102,180,3.2500,Order of Arms is a simulation game of battles ...,1,2,...,False,False,False,False,False,False,2364,9,148,205
498,12288,1066,Zargos,1990,5.54864,120,2.6923,Principle of the game:&#10;The players' goal i...,2,6,...,False,False,False,False,False,False,1006,11,40,102


Use TFIDF Vectorizer

In [120]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [121]:
with open('stopwords.txt') as f:
    sw = f.read().split()

In [124]:
transformer = TfidfVectorizer(min_df=.05, max_df=.3, stop_words=sw)

In [136]:
vectorMat =transformer.fit_transform(trainingdata['descriptions'].str.lower()) 
testvecMat = transformer.transform(testingdata['descriptions'].str.lower()) 

feature_names = transformer.get_feature_names_out()


trainingVec = pd.DataFrame(vectorMat.toarray(), columns=feature_names)
testingVec = pd.DataFrame(testvecMat.toarray(), columns=feature_names)

In [146]:
trainingFinal = trainingdata.join(trainingVec)
testingFinal = testingdata.join(testingVec)

In [ ]:
#trainingFinal.to_csv('cleantraining.csv')
#testingFinal.to_csv('cleantesting.csv')

In [145]:
temp

,Unnamed: 0,id,name,yearpublished,bayesaverage,descriptions,minplayers,maxplayers,minplaytime,maxplaytime,...,whether,whoever,win,winner,winning,wins,without,work,world,worth
0,0,224517,Brass: Birmingham,2018,8.41324,Brass: Birmingham is an economic strategy game...,2,4,60,120,...,0.000000,0.0,0.131079,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0
1,1,161936,Pandemic Legacy: Season 1,2015,8.37590,Pandemic Legacy is a co-operative campaign gam...,2,4,60,60,...,0.000000,0.0,0.099808,0.000000,0.0,0.0,0.000000,0.000000,0.204075,0.0
2,2,174430,Gloomhaven,2017,8.34456,Gloomhaven is a game of Euro-inspired tactica...,1,4,60,120,...,0.140170,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.120809,0.301269,0.0
3,3,342942,Ark Nova,2021,8.33908,"In Ark Nova, you will plan and design a modern...",1,4,90,150,...,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.103402,0.0
4,4,233078,Twilight Imperium: Fourth Edition,2017,8.23710,Twilight Imperium (Fourth Edition) is a game o...,3,6,240,480,...,0.096281,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
759,27181,2136,Pachisi,400,4.70614,"Pachisi, the national game of India, dates bac...",2,6,30,30,...,0.000000,0.0,0.000000,0.166361,0.0,0.0,0.000000,0.000000,0.000000,0.0
760,27182,2407,Sorry!,1929,4.68837,Slide Pursuit Game&#10;&#10;Race your four gam...,2,4,30,30,...,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.107041,0.000000,0.000000,0.0
761,27188,3737,Operation,1965,4.54690,Operation is a dexterity game in which you mu...,1,6,10,10,...,0.000000,0.0,0.000000,0.201741,0.0,0.0,0.000000,0.000000,0.000000,0.0
762,27190,2921,The Game of Life,1960,4.36687,This game attempts to mirror life events many ...,2,6,60,60,...,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.142866,0.000000,0.000000,0.0
